In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
df = pd.read_csv('..//..//dataset//final_dataset.csv')  


X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

X_scaled = X

X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [ ]:
def build_drnn_lstm():
    model = Sequential()
    
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    model.add(LSTM(units=100, activation='relu',return_sequences=True))
    model.add(LSTM(units=100, ))
    
    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [ ]:
model = build_drnn_lstm()
import time
batch_size1 = X_train.shape[0]  # Use full dataset per batch to match experimental setup
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=batch_size1//3,
    verbose=1,
    validation_split=0.2
)
end_time = time.time()

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - loss: 0.6924 - mae: 0.4996 - mse: 0.2496 - val_loss: 0.6910 - val_mae: 0.4989 - val_mse: 0.2489
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 0.6908 - mae: 0.4988 - mse: 0.2488 - val_loss: 0.6890 - val_mae: 0.4978 - val_mse: 0.2479
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 0.6887 - mae: 0.4976 - mse: 0.2478 - val_loss: 0.6861 - val_mae: 0.4962 - val_mse: 0.2465
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6857 - mae: 0.4960 - mse: 0.2463 - val_loss: 0.6823 - val_mae: 0.4939 - val_mse: 0.2446
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 0.6817 - mae: 0.4935 - mse: 0.2443 - val_loss: 0.6772 - val_mae: 0.4904 - val_mse: 0.2421


In [ ]:
total_training_time = end_time - start_time  # in seconds
epochs = 5 # e.g., 50
steps_per_epoch = 3# e.g., 2000

total_steps = epochs * steps_per_epoch

time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
time_per_step_us = time_per_step_ms * 1000

time_per_epoch = total_training_time / epochs

print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 15
Time per Step: 0.808849 sec (808.85 ms)
Time per Step: 808849.03 µs
Time per Epoch: 2.43 sec


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.16f}')


In [ ]:
import matplotlib.pyplot as plt

# Visualizing the loss, MSE, and MAE
plt.figure(figsize=(15, 5))

# Loss plot
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# MSE plot
plt.subplot(1, 3, 2)
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.title('Mean Squared Error')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# MAE plot
plt.subplot(1, 3, 3)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()
